# Linear Regression Basics

Regressão Linear (R.L) é uma técnica de modelação preditiva para prever uma variável de resposta numérica (Nº Real) baseada em uma ou mais variáveis explicativas.

No caso da Regressão Linear com uma única variável explicativa, utilizamos a combinação linear a seguir:

respose = intercept + constant x explanatory, 

ou seja, uma equação linear (y(x) = a + bx).

O modelo é construído para se ajustar a uma linha que minimize os erros ou resíduos. O resultado é buscar por um fit linear que melhor ajustam os dados. 

Vamos construir dados que relacionam o quanto um carro gasta de combustível dependendo de seu peso (mpg x weight). 



In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as stats

matplotlib.style.use('ggplot')

In [9]:
mtcars = pd.read_csv("../input/mtcars/mtcars.csv")

mtcars.plot(kind = "scatter",
           x = "wt",
           y = "mpg",
           figsize = (9,9),
           color = "black")



O plot dos pontos mostram que eles tem dependencias aproximadamente liner, sem apresentar muitos outliers, sugerindo que uma regressão linear funcionara bem. 

A lib. Scikit-learn contem um monte de funções que podem ser utilizadas em modelos preditivos, vamos importar a regressão linear dela.

In [10]:
from sklearn import linear_model

In [16]:
regression_model = linear_model.LinearRegression() # aqui estamos começando o modelo  (Ele sempre precisa ser inicializado)

regression_model.fit( X = pd.DataFrame(mtcars["wt"]),
                    y = mtcars["mpg"])

print(regression_model.intercept_)
print(regression_model.coef_)

Os outputs acima mostram que o fit que o modelo foi é:


mpg = 37.2851261673420  -5.34447157 x wt

Podemos ter uma ideia do quanto a variancia  da variavel de resposta (mpg) é explicado por este fit usando model.score() function.

In [17]:
regression_model.score( X = pd.DataFrame(mtcars["wt"]),
                       y = mtcars["mpg"]) 

Neste caso, vemos que o peso do carro explica 75% da variancia da variável de resposta (mpg). Este valor é chamado de "R-squared" e tem range entre 0 e 1. R-squared é baseado nos resíduos: diferenças entre o que o modelo prevê para cada ponto de dados e o valor real de cada ponto. Podemos extrair os resíduos do modelo fazendo uma previsão com o modelo sobre os dados e depois subtraindo o valor real de cada previsão

In [22]:
train_prediction = regression_model.predict(X = pd.DataFrame(mtcars["wt"])) 

residuals = mtcars["mpg"] - train_prediction
residuals.describe()

R-squared é calculado por 1 - SSresidual/SStoal, que serão definidos a seguir:

In [28]:
SSresiduals = (residuals**2).sum()
SStotal = ((mtcars["mpg"] - mtcars["mpg"].mean())**2).sum()

R-squared = 1 - SSresiduals/SStotal

Vamos fazer agora um scatterplot dos dados e do modelo junto para a visualização de como este é um bom fit para os dados.

In [40]:
mtcars.plot(kind = "scatter",
           x =  "wt",
           y = "mpg",
           figsize = (9,9),
            color = "black",
           xlim = (0,7))

plt.plot(mtcars["wt"], train_prediction, color = "blue")

Os outliers podem ter uma grande influência nos modelos de regressão linear: uma vez que a regressão trata da minimização dos resíduos quadrados, os grandes resíduos têm uma influência desproporcionalmente grande no modelo. Assim, é preciso ter cuidados com os outliers e em alguns casos usamos métodos para a remoçao deles. Vamos acrescentar um outliers - um carro super pesado e eficiente em termos de combustível - e traçar um novo modelo de regressão:

In [73]:
mtcars_subset = mtcars[["mpg","wt"]]

super_car = pd.DataFrame({"mpg" : 50, "wt":10}, index = ["super"])

new_cars = mtcars_subset.append(super_car)


regression_model = linear_model.LinearRegression()
regression_model.fit(X = pd.DataFrame(new_cars["wt"]), 
                     y = new_cars["mpg"])

train_prediction2 = regression_model.predict(X = pd.DataFrame(new_cars["wt"]))

new_cars.plot(kind = "scatter",
             x = 'wt',
             y = 'mpg',
             color = "black",
             figsize = (9,9),
             xlim = (1,11), ylim = (10,52))

plt.plot(new_cars["wt"], train_prediction2, color = "blue")



Neste plot, podemos ver como um único outiliers pode estragar totalmente o modelo. Claro que este é um outilier extremo e pelo plot poderiamos removelo e voltar a um bom modelo.

Num modelo de regressão linear bem comportado, gostaríamos que os resíduos fossem distribuídos de forma mais ou menos normal. Ou seja, gostaríamos de uma distribuição de erros fosse mais ou menos uniforme acima e abaixo da linha de regressão. Podemos investigar a normalidade dos resíduos com um gráfico Q-Q (quantile-quantile). 

In [75]:
plt.figure(figsize = (9,9))
stats.probplot(residuals, dist = "norm", plot = plt)

Quando os resíduos são normalmente distribuídos, tendem a se concentrar ao longo da linha reta na plot Q-Q. Neste caso, os resíduos parecem seguir um padrão um pouco não linear: os resíduos são curvados um pouco para longe da linha de normalidade em cada extremidade. Isto é uma indicação de que uma simples linha reta pode não ser suficiente para descrever completamente a relação entre peso e mpg.